**Linear regression:** https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho 

**Logistic regression:** https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009/code 

**Neural network:** https://www.kaggle.com/competitions/titanic/overview 

**K-Means:** https://www.kaggle.com/datasets/camnugent/california-housing-prices 

In [2]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import random as rd

## 1- Linear Regression

**Linear Regression** is a fundamental statistical and machine learning technique used for modeling the relationship between a *dependent* variable (also called the target or outcome) and one or more *independent* variables (also called predictors or features).

It assumes a linear relationship between the variables and aims to find the best-fitting linear equation that can predict the dependent variable based on the values of the independent variables.
